In [2]:
%matplotlib inline
import os 
import pandas as pd
import matplotlib.image as mpimg
# import shutil
import matplotlib.pyplot as plt
import pickle
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from scipy.misc import imresize

In [3]:
#Load images
imagequality = 50 # This is the % reduction of image quality. 20 is equal to reducing the pixels to 20%

phototexture    = []

folder=os.getcwd()+'/photos/'+'texture/'
texture_dict = {}

filelist = glob.glob(folder+'/*.png')
textureimages = np.array([imresize(np.array(Image.open(fname).convert('RGBA')),imagequality) for fname in filelist])
texturefiles = np.array([np.array(fname.split('/')[-1].replace('.png','')) for fname in filelist])
texturefiles = list(map(int, texturefiles))

In [3]:
texture = pd.read_csv('texture.csv')
texture = texture.drop(['Unnamed: 0'],axis=1)
texture['texture'] = texture.texture.str.replace(',','.')
texture['texture'] = pd.to_numeric(texture['texture'])
texture['inx']   = texture.index

texturefiles = pd.DataFrame(texturefiles,columns=['inx'])
texturedata = texturefiles.merge(texture,on='inx')

errorimages = [1126,1132,1143]
errorindex  = texturedata[texturedata.inx.isin(errorimages)].index.values
texturedata = texturedata.drop(errorindex)

texturedata = texturedata.drop(['filename','empresa','gravel_size','image','inx'],axis=1)
texturedata['category'] = pd.cut(texturedata.texture,bins=[-0.01,165,275,350,460,20000],labels=[0, 1, 2, 3, 4])

textureimages = np.delete(textureimages,errorindex,0)

In [4]:
#Binary data
np.save('textureimagequality50.npy', textureimages)

In [5]:
texturedata.to_csv("texturedataprepared.csv")